## FINDING THE BEST APP TYPE: ##
our project consists in analyzing andriod and ios stores data with the purpose of finding out whats the best category to make a profitable free in_app ad app. which category would attract more users?

In [6]:
#first lets open the files #

opened_ios = open('./Downloads/AppleStore.csv', encoding = 'utf8')
opened_android = open('./Downloads/googleplaystore.csv', encoding = 'utf8')

#then we need to import the csv reader to read it and store in the lists

from csv import reader
readed_ios = reader(opened_ios)
readed_android = reader(opened_android)

#once they are readed, we can store it inside lists

ios_data = list(readed_ios)
android_data = list(readed_android)


lets create a function that helps us explore the data

In [56]:
#this function will help in printing rows repeatedly in a readable way
def explore_data(dataset, start, end, rows_and_columns= False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')  # adds a new empty line after each row to help readability #
        
    if rows_and_columns == True:
        print('Number of rows', len(dataset))
        print('Number of columns', len(dataset[0]))
        


lets make some short analysis of the dimensions and columns of both datasets. the meaning of the columns of google play store are not described by the dataset owner, but from the apple store dataset.

In [58]:
print('Google Play Store apps data columns and dimensions \n')
explore_data(android_data, 0, 1, True)
print('\n', '-'*60, '\n')

print('Apple Store apps columns and dimensions \n')
explore_data(ios_data, 0, 1, True)

Google Play Store apps data columns and dimensions 

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Number of rows 10841
Number of columns 13

 ------------------------------------------------------------ 

Apple Store apps columns and dimensions 

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Number of rows 7198
Number of columns 16


looking at the columns of our dataset we can identify some important information that we will use to answer our main question like:
Google Play Store = 'App', 'category', 'Rating', 'Installs', 'Type', 'Price', 'Genres'
App Store = 'id', 'price', 'rating_count_tot', 'user_rating', 'cont_rating', 'prime_genre'

now we will store the columns names in a different list so that we can use it for future reference

In [60]:
# android column names
columns_android = android_data[0]

# ios column names

columns_ios = ios_data[0]


print('Ándroid columns\n', columns_android)
print('\niOS columns\n', columns_ios)

Ándroid columns
 ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']

iOS columns
 ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


## 2. CLEANING DATA ##

before any data analysis is carried out, it is extremely important that data assessment is carried out. data assessment include data cleaning, removing duplicates, checking for accuracy of data and so on.

## 2.1 Checking for wrong data ##

checking the discussion forum of our dataset we can see theres an erorr in row 10472(actually 10473). we print it to be sure, then we'll delete it.

In [10]:
android_data[10473] # this particular row does not have category in it, so have to delete it because it is not in line with other dataset

['Life Made WI-Fi Touchscreen Photo Frame',
 '1.9',
 '19',
 '3.0M',
 '1,000+',
 'Free',
 '0',
 'Everyone',
 '',
 'February 11, 2018',
 '1.0.19',
 '4.0 and up']

In [11]:
del android_data[10473] #deleting row 10473 from the dataset

#to check if its gone
explore_data(android_data, 0, 1, True)



['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Number of rows 10841
Number of columns 13


checking for other wrong data

In [12]:
#looking at instagram entries

for app in android_data:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


fully aware that there is duplicate, we will create a function that gives us the number of duplicates and store the name of duplicates, so that we can remove the duplicates

In [13]:
#first we will store the duplicates values into list
duplicated = []
unique = []

for app in android_data:
    name = app[0]
    if name in unique:
        duplicated.append(name)
    else:
        unique.append(name)
        
#then we can see how many entries each list have
print(len(duplicated))
print(len(unique))

1181
9660


with that in mind, we wont remove the duplicates randomly. looking at the 'instagram' entries we printed, we can see the 4th column has the number of reviews, thinking about it logically we can see that the biggest number reflects the newer entry. this is the entry we are going to keep.

In [14]:
#creating a dictionary to store name and maximum
reviews_max = {}

#looping through the data to populate the dictionary
for app in android_data[1:]:  #excluding the header column
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
#seeing if everything went fine
print('size of reviews_max dictionary', len(reviews_max))
    

size of reviews_max dictionary 9659


now that we have created this dictionary with entries(name of the app and amount of reviews), we can go back to our main data set and start to remove the ones that are not there.

the loop consists in checking two things:
1) if the number of reviews from our main dataset is equal to the one stored in our dictionary with the max review number, showing that this entry is the newest.
2)if the entry has already been added(it checks if it already exists in the already_added list).

once the entry has the same number of review and hasn't been added, it will be stored in our new list created to recieve the cleaned data android_clean.

finally we print the length of the list to see if everything went fine (it has to be the same length as th reviews_max that we did above

In [15]:
# Creating a list to store our cleaned data
android_clean = []

# Creating a help list to store the app names
already_added = []

# looping through the data to clean it
for app in android_data[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name) 
print('size of android_clean list', len(android_clean))

size of android_clean list 9659


## LOOKING FOR DUPLICATES IN APPLE STORE ##

using a similiar approach we are going to insert the unique ids in the list unique and the repeated ones in the duplicated. then we'll see the length of them

In [16]:
# creating the list

unique = []
duplicated = []

# adding values to the empty list above:
for app in ios_data:
    app_id = app[0]
    if app_id in unique:
        duplicated.append(app_id)
    else:
        unique.append(app_id)
        
#checking to see how it looks like
print('unique values: {} \nduplicated values: {}'.format(len(unique), len(duplicated)))

unique values: 7198 
duplicated values: 0


As we can see there are no duplicated values in the dataset. but other issues comes, when we think that our company wants to develop an app to the english speaking audience, we need to remove the apps that are not in english. and in both datasets we see this type of entry.

## REMOVING THE NON ENGLISH APPS ##



In [17]:
# printing non english apps in the ios dataase
print('Apple store non-english apps: \n \n{} \n{}\n'.format(ios_data[814][1], ios_data[6732][1]))

#printing non-english app in the android database
print('\nGoogle Play Store non-english apps: \n \n{} \n{}'.format(android_clean[4412][0],android_clean[7940][0]))

Apple store non-english apps: 
 
爱奇艺PPS -《欢乐颂2》电视剧热播 
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


Google Play Store non-english apps: 
 
中国語 AQリスニング 
لعبة تقدر تربح DZ


in order to remove non english apps that have characters that are not contained in English language. we can know that by the index of every letter(this is found with the ord() built_in function), the characters from the english alphabet are in the range of 0 to 127 (according to the ASCII)

with that in mind, we are going to create a function that sees if the app name has non-english characters GoTM


In [25]:
# loops over the chars and see if they have any non-english characters
def english_char(a_string):
    for char in a_string:
        if ord(char) > 127:
            return False
    return True

# lets check some values
print('Instagram is engish-language app. \t\t\t\t', english_char('Instagram'))
print('爱奇艺PPS -《欢乐颂2》电视剧热播 is an english app.\t\t', english_char('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print('Docs To Go™ free office Suite is an english app.\t\t', english_char('Docs To Go™ free office Suite'))
print('Instachat 😜 is an english-language app.\t\t\t', english_char('Instachat 😜'))

Instagram is engish-language app. 				 True
爱奇艺PPS -《欢乐颂2》电视剧热播 is an english app.		 False
Docs To Go™ free office Suite is an english app.		 False
Instachat 😜 is an english-language app.			 False


the reason that our function returns 'Instachat 😜' and 'Docs To Go™ free office Suite' as non english language apps is because of characters like TM fall outside the ASCII range and have numbers over 127.

we cannot exclude this values from our dataset, since it is really useful. To fix it in some way, we will allow our strings to have maximum 3 special characters. abaelow we will rewrite our function to do that.

In [26]:
# loops over the chars and see if they have more than 3 non-english characters
def english_char(a_string):
    special_char = 0
    for char in a_string:
        if ord(char) > 127:
            special_char += 1
    if special_char >= 3:
        return False
    else:
        return True
    
#let's check some values
print('Instagram is engish-language app. \t\t\t\t', english_char('Instagram'))
print('爱奇艺PPS -《欢乐颂2》电视剧热播 is an english app.\t\t', english_char('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print('Docs To Go™ free office Suite is an english app.\t\t', english_char('Docs To Go™ free office Suite'))
print('Instachat 😜 is an english-language app.\t\t\t', english_char('Instachat 😜'))

Instagram is engish-language app. 				 True
爱奇艺PPS -《欢乐颂2》电视剧热播 is an english app.		 False
Docs To Go™ free office Suite is an english app.		 True
Instachat 😜 is an english-language app.			 True


Now we'll use our new function to filter the apps that are in english language and make our dataset cleaner.

In [28]:
#cleaning the android data
play_store_data = []
for app in android_clean[1:]:
    name = app[0]
    if name not in android_data:
        if english_char(name) == True:
            play_store_data.append(app)
            
a = len(android_clean)
b = len(play_store_data)

print('the new clean dataset has {} entries and the older had {}, {} entries were removed'.format(b, a, a-b))

the new clean dataset has 9596 entries and the older had 9659, 63 entries were removed


In [30]:
# cleaning the ios_data
app_store_data = []

for app in ios_data[1:]:
    app_id = app[0]
    if app_id not in app_store_data:
        if english_char(app_id) == True:
            app_store_data.append(app)
            
a = len(ios_data)
b = len(app_store_data)
print('the new clean dataset has {} entries and the older had {}, {} entries were removed'.format(b, a, a-b))

the new clean dataset has 7197 entries and the older had 7198, 1 entries were removed


Now that we made it we have just another thing to do. Since our company wants to create a free app with ads in it, we need to separate the free apps from the non-free apps. This is the last step of our data cleaning process

## 2.5 Separating the free from the non-free apps ##

First we will check in which column the prices are in both datasets

In [32]:
print(columns_android)
print('\n')
print(columns_ios)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


From the output above, we can see we need the 8th column of android data and the 5th of the ios data. Knowing this, we will start the operation

In [39]:
#creating list to store free apps
free_android_apps = []
free_ios_apps = []

for app in play_store_data:
    price = float(app[7].replace('$', ''))
    if price == 0.00:
        free_android_apps.append(app)
        
for app in app_store_data:
    price = float(app[4].replace('$', ''))
    if price == 0.00:
        free_ios_apps.append(app)
        
print('number of free android apps is', len(free_android_apps))
print('number of all android apps is', len (play_store_data))
print('\nnumber of free ios apps is', len(free_ios_apps))
print('number of all ios apps is', len(app_store_data))

number of free android apps is 8847
number of all android apps is 9596

number of free ios apps is 4056
number of all ios apps is 7197


## 3 Analyzing data ##

Thinking about the primary objective(launch an app in App Store and Play Store), we need to evaluate both platforms. To minimize the risk the validation strategy for an app idea is comprised of three steps(as said in our DataQuesr course):

     1. Build a minimal Android version of the app, and add it to GOogle play
     2. If the app has a good response from users, we develop it further
     3. If the app is profitable after six months, we build an IOS version of the app and add it to the App store
     
To do so, we are going to use the genres column (index 9) and category column(index 1) in Google play store dataset and the prime_genre column (index 11) in our App Store dataset.

To analyze it this way, we are going to make two functions: one to make the frequency tables with percentages and another to put the percentages in descending order.

In [42]:
# making a function that creates a frequency table that shows percentages
def freq_table(dataset, index):
    frequency_table = {}
    total = len(dataset)
    for row in dataset:
        entry = row[index]
        if entry in frequency_table:
            frequency_table[entry] += 1
        else:
            frequency_table[entry] = 1
            
    for key in frequency_table:
        frequency_table[key] /= total/100
        frequency_table[key] = round(frequency_table[key], 4)
        
    return frequency_table

#check to see whether the first entries of dictionary will print
print(list((freq_table(free_android_apps, 1)).items())[:10])

[('ART_AND_DESIGN', 0.633), ('AUTO_AND_VEHICLES', 0.9269), ('BEAUTY', 0.5991), ('BOOKS_AND_REFERENCE', 2.1363), ('BUSINESS', 4.6004), ('COMICS', 0.6104), ('COMMUNICATION', 3.2327), ('DATING', 1.865), ('EDUCATION', 1.1642), ('ENTERTAINMENT', 0.9608)]


Above we have created the dictionary and it worked. to slice it we have printed a slice of a list made with its items(dictionaries cannot be sliced), we didnt change the dictionary, we just printed it in a different way. lets do the same thing with the App store data.

In [43]:
# making a function that displays the percentages in a descending order
# this function was already made by Dataquest, they claimed that this is easier with other resources

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [45]:
# display table for free_android_apps genre

display_table(free_android_apps, 9)

Tools : 8.4435
Entertainment : 6.0812
Education : 5.3577
Business : 4.6004
Productivity : 3.8996
Lifestyle : 3.877
Finance : 3.7075
Medical : 3.5379
Sports : 3.4588
Personalization : 3.3232
Communication : 3.2327
Action : 3.0971
Health & Fitness : 3.0858
Photography : 2.9502
News & Magazines : 2.8032
Social : 2.6676
Travel & Local : 2.3285
Shopping : 2.2494
Books & Reference : 2.1363
Simulation : 2.0459
Dating : 1.865
Arcade : 1.8424
Video Players & Editors : 1.7746
Casual : 1.7633
Maps & Navigation : 1.3903
Food & Drink : 1.2434
Puzzle : 1.1303
Racing : 0.9947
Role Playing : 0.9382
Libraries & Demo : 0.9382
Auto & Vehicles : 0.9269
Strategy : 0.9156
House & Home : 0.8025
Weather : 0.7912
Events : 0.7121
Adventure : 0.6669
Comics : 0.5991
Beauty : 0.5991
Art & Design : 0.5878
Parenting : 0.4973
Card : 0.4521
Trivia : 0.4182
Casino : 0.4182
Educational;Education : 0.3956
Board : 0.3843
Educational : 0.373
Education;Education : 0.3391
Word : 0.26
Casual;Pretend Play : 0.2374
Music : 0.20

In [47]:
# display free_android_apps categories

display_table(free_android_apps, 1)

FAMILY : 18.9443
GAME : 9.6982
TOOLS : 8.4548
BUSINESS : 4.6004
PRODUCTIVITY : 3.8996
LIFESTYLE : 3.8883
FINANCE : 3.7075
MEDICAL : 3.5379
SPORTS : 3.391
PERSONALIZATION : 3.3232
COMMUNICATION : 3.2327
HEALTH_AND_FITNESS : 3.0858
PHOTOGRAPHY : 2.9502
NEWS_AND_MAGAZINES : 2.8032
SOCIAL : 2.6676
TRAVEL_AND_LOCAL : 2.3398
SHOPPING : 2.2494
BOOKS_AND_REFERENCE : 2.1363
DATING : 1.865
VIDEO_PLAYERS : 1.7972
MAPS_AND_NAVIGATION : 1.3903
FOOD_AND_DRINK : 1.2434
EDUCATION : 1.1642
ENTERTAINMENT : 0.9608
LIBRARIES_AND_DEMO : 0.9382
AUTO_AND_VEHICLES : 0.9269
HOUSE_AND_HOME : 0.8025
WEATHER : 0.7912
EVENTS : 0.7121
PARENTING : 0.6556
ART_AND_DESIGN : 0.633
COMICS : 0.6104
BEAUTY : 0.5991


In [49]:
# display table for free_ios_apps prime_genres

display_table(free_ios_apps, 11)

Games : 55.646
Entertainment : 8.2347
Photo & Video : 4.1174
Social Networking : 3.5256
Education : 3.2544
Shopping : 2.9832
Utilities : 2.6874
Lifestyle : 2.3176
Finance : 2.071
Sports : 1.9477
Health & Fitness : 1.8738
Music : 1.6519
Book : 1.6272
Productivity : 1.5286
News : 1.43
Travel : 1.3807
Food & Drink : 1.0602
Weather : 0.7643
Reference : 0.4931
Navigation : 0.4931
Business : 0.4931
Catalogs : 0.2219
Medical : 0.1972


relevant questions to answer about our data:

1) What is the most common genre? What is the runner-up?
2) What patterns can we see?
3) What is the general impression- are most apps designed for practical purposes or more for entertainment?
4) Could we recommend and app profile based on the information we have so far? Does a bi amount of apps in that genre imply a large number of users?

With that in mind we can see that App Store has a big amount of entertainment apps, on the other hand Play Store is more balanced, having productivity, education and business in the top 5. But this data isnt enough to set a strategy. we need to know how many people have downloaded this apps.
The Play Store dataset has a column called intalls(index5) that gives us already this information. App Store dataset does not have this information, the solution is to look at the total number of user rating in the rating_count_tot(index 5), since we don't have a better option. Next we are going to calculate the average number of user ratings per app genre in App Store

In [50]:
# making a frequency table for prime_genre
prime_genre_freq = freq_table(free_ios_apps, 11)

for genre in prime_genre_freq:
    total = 0
    len_genre = 0
    for app in free_ios_apps:
        genre_app = app[11]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_ratings = round(total/len_genre, 4)
    genre_ratings = [genre, avg_ratings]
    print(genre, genre_ratings)

Social Networking ['Social Networking', 53078.1958]
Photo & Video ['Photo & Video', 27249.8922]
Games ['Games', 18924.689]
Music ['Music', 56482.0299]
Reference ['Reference', 67447.9]
Health & Fitness ['Health & Fitness', 19952.3158]
Weather ['Weather', 47220.9355]
Utilities ['Utilities', 14010.1009]
Travel ['Travel', 20216.0179]
Shopping ['Shopping', 18746.6777]
News ['News', 15892.7241]
Navigation ['Navigation', 25972.05]
Lifestyle ['Lifestyle', 8978.3085]
Entertainment ['Entertainment', 10822.9611]
Food & Drink ['Food & Drink', 20179.093]
Sports ['Sports', 20128.9747]
Book ['Book', 8498.3333]
Finance ['Finance', 13522.2619]
Education ['Education', 6266.3333]
Productivity ['Productivity', 19053.8871]
Business ['Business', 6367.8]
Catalogs ['Catalogs', 1779.5556]
Medical ['Medical', 459.75]


   ## looking at the data we have analyzed so far, we can see that in App Store good ideas of apps would be in Social Networking, reference, weather or music field, since the four of them have more than 40 thousand ratings(indicating a lot of downloads) and they are not saturated: Music apps are 1.65% of our dataset, weather 0.76%, Social Networking 3.53% and reference 0.49% ##

Now we need to evaluate the amount of installs of the apps from the Play Store. They do not have an exact number for each apps, so we are goin to work with have. Once we take this values, we will treat them and print so we can analyze it.

In [52]:
# making a frequency table for category (playstore)
category_freq = freq_table(free_android_apps, 1)

for category in category_freq:
    total = 0
    len_category = 0
    for app in free_android_apps:
        category_app = app[1]
        if category_app == category:
            n_ratings = float((app[5].replace('+', '')).replace(',',''))
            total += n_ratings
            len_category += 1
    avg_ratings = round(total/len_category, 4)
    category_ratings = [category, avg_ratings]
    print(category, avg_ratings)

ART_AND_DESIGN 2021626.7857
AUTO_AND_VEHICLES 647317.8171
BEAUTY 513151.8868
BOOKS_AND_REFERENCE 8814199.7884
BUSINESS 1712290.1474
COMICS 832613.8889
COMMUNICATION 38590581.0874
DATING 854028.8303
EDUCATION 1833495.1456
ENTERTAINMENT 11640705.8824
EVENTS 253542.2222
FINANCE 1387692.4756
FOOD_AND_DRINK 1924897.7364
HEALTH_AND_FITNESS 4188821.9853
HOUSE_AND_HOME 1360598.0423
LIBRARIES_AND_DEMO 638503.7349
LIFESTYLE 1446158.2238
GAME 15544014.5105
FAMILY 3695641.8198
MEDICAL 120550.6198
SOCIAL 23253652.1271
SHOPPING 7036877.3116
PHOTOGRAPHY 17840110.4023
SPORTS 3650602.2767
TRAVEL_AND_LOCAL 13984077.7101
TOOLS 10830251.9706
PERSONALIZATION 5201482.6122
PRODUCTIVITY 16787331.3449
PARENTING 542603.6207
WEATHER 5145550.2857
VIDEO_PLAYERS 24727872.4528
NEWS_AND_MAGAZINES 9549178.4677
MAPS_AND_NAVIGATION 4049274.6341


In [53]:
# making a frequency table for category/genre column(Play Store)
genre_freq = freq_table(free_android_apps, 9)

for genre in genre_freq:
    total = 0
    len_genre = 0
    for app in free_android_apps:
        genre_app = app[9]
        if genre_app == genre:
            n_ratings = float((app[5].replace('+', '')).replace(',',''))
            total += n_ratings
            len_genre += 1
    avg_ratings = round(total/len_genre, 4)
    genre_ratings = [genre, avg_ratings]
    print(genre, avg_ratings)


Art & Design 2163482.6923
Art & Design;Creativity 285000.0
Auto & Vehicles 647317.8171
Beauty 513151.8868
Books & Reference 8814199.7884
Business 1712290.1474
Comics 847380.1887
Comics;Creativity 50000.0
Communication 38590581.0874
Dating 854028.8303
Education 550185.443
Education;Creativity 2875000.0
Education;Education 4759517.0
Education;Pretend Play 1800000.0
Education;Brain Games 5333333.3333
Entertainment 5602792.7751
Entertainment;Brain Games 3314285.7143
Entertainment;Creativity 4000000.0
Entertainment;Music & Video 6413333.3333
Events 253542.2222
Finance 1387692.4756
Food & Drink 1924897.7364
Health & Fitness 4188821.9853
House & Home 1360598.0423
Libraries & Demo 638503.7349
Lifestyle 1421219.9096
Lifestyle;Pretend Play 10000000.0
Card 3815462.5
Arcade 23028171.411
Puzzle 8302861.91
Racing 15910645.6818
Sports 4611701.5784
Casual 19569221.6026
Simulation 3475484.0884
Adventure 4158764.7458
Trivia 3475712.7027
Action 12467105.6204
Word 9094458.6957
Role Playing 3965645.4217
St

## looking at the data we have printed, we can see that the most installed apps that are not a big part of the total apps are:

1)Video players more than 24million downloads and 1.79%
2) Entertainment with more than 11million downloads and 0.96% of total apps
3) Books and references with more than 8million downloads and 2.13% of total apps
4) Weather with more than 5million downloads and 0.79% of total apps
5) Music with more than 9million downloads and 0.30%(approximately) of total apps

## keeping in mind that we want an app for both platforms(play store and app store), books, references, music app are good choices, since they are downloaded alot and have a small share in the total number of apps in both markets. To refine our decision, we are going to take a look in these specific genres to see if there's something interesting there

# Books, references and Music apps analysis

In [54]:
# making a list with all book and reference android apps
books_android = []
for app in free_android_apps:
    category = app[1]
    if category == 'BOOKS_AND_REFERENCE':
        books_android.append(app)
        
#making a list with all music android apps
music_android = []
for app in free_android_apps:
    genre = app[9]
    if 'Music' in genre:
        music_android.append(app)
        
#making list with all book and reference ios apps

books_ios =[]
for app in free_ios_apps:
    category = app[11]
    if category == 'Reference':
        books_ios.append(app)
    elif category == 'Book':
        books_ios.append(app)
        
#making a list with all music ios apps
music_ios = []
for app in free_ios_apps:
    category = app[11]
    if category == 'Music':
        music_ios.append(app)

print('lenth of books_android (and references)', len(books_android))
print('length of music_android:', len(music_android))
print('\n')
print('length of books_ios (and references):', len(books_ios))
print('length of music_ios', len(music_ios))

lenth of books_android (and references) 189
length of music_android: 48


length of books_ios (and references): 86
length of music_ios 67
